# EXPLORER DATA SET 

1. 서울/외식&카페 회사정보 뽑기
2. 외식 결제건수 150 이상 Unique user 100명씩 뽑기(payment칼럼으로 금액, 결제건수, 상점수 필요)
 - 1번조건 만족하면 df저장하기 
3. ratio1,2 컬럼 추가하기

In [25]:
import pymysql, pandas as pd

conn = pymysql.connect(host='133.186.143.151', port=13306, user='eums', password='eums00!q', db='eums-shared', charset='utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)
curs.execute("select ID AS COMPANY_ID, CO_NAME, ADDR,LATITUDE, LONGITUDE FROM MEUMS_COMPANY WHERE CATE_CODE in ('C12','C11') and status in (0,1) ;") ##외식, 카페/디저트
restaurant = pd.DataFrame( curs.fetchall() )[['COMPANY_ID','CO_NAME','ADDR','LATITUDE','LONGITUDE']]
conn.close()
print(restaurant.shape)
restaurant.head()

(1086294, 5)


,COMPANY_ID,CO_NAME,ADDR,LATITUDE,LONGITUDE
0,13,강스 의정부식부대찌개쌈밥,서울 성동구 행당동 317-134,37.55799780042937,127.03060766662924
1,15,(구)서울떡집,인천광역시 중구 신포동 13,37.4711881,126.6274865
2,19,구마장동뒷고기,경북 포항시 북구 장성동 976-2,36.0693676,129.3729044
3,34,(북면)한식전문점,경기도 가평군 북면 목동리 840-1,37.88570985,127.5493877
4,62,(사단)공정무역인천광장,인천광역시 부평구 부평4동 534-1,37.5008313,126.7249104


In [26]:
#서울지역 외식카페 필터링
rstt_seoul =restaurant[restaurant['ADDR'].str.contains('서울')]
print(rstt_seoul.shape)
rstt_seoul.head()

(221890, 5)


,COMPANY_ID,CO_NAME,ADDR,LATITUDE,LONGITUDE
0,13,강스 의정부식부대찌개쌈밥,서울 성동구 행당동 317-134,37.55799780042937,127.03060766662924
10,208,(사단)서울클럽,서울특별시 중구 장충동2가 208,37.5532549,127.0038366
15,493,(서백자)명인간장게장(삼성점),서울특별시 강남구 삼성1동 153-19,37.5101339,127.0568883
16,494,어뎅맛뎅,서울특별시 종로구 홍지동 54,37.6008889,126.9557351
22,626,(유한)산들농산,서울특별시 송파구 가락동 600,37.49342264,127.1111402


In [27]:
company_id = rstt_seoul.COMPANY_ID.unique().tolist()
len(company_id)

221890

In [28]:
#company_id=company_id[0:2] 
rstt_sales= pd.DataFrame([])
len(company_id)

221890

In [29]:
conn = pymysql.connect(host='192.168.0.50', user='eums', password='eums00!q', db='eums', charset='utf8')
#conn = pymysql.connect(host='133.186.135.21', user='eums', password='eums00!q', db='eums', charset='utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)

for i, value in enumerate(company_id[0:100000]) :
    curs.execute("select EPOPCON_ID, COMPANY_ID,DEAL_DT,CARD_NAME,PAYMENT,REG_DT from MEUMS_COMP_REVISIT_HIS WHERE COMPANY_ID = "
                 +str(value)+" AND deal_dt between '2016-01-01 00:00:00' and '2018-03-31 23:59:59';")
    try :
        bb = pd.DataFrame( curs.fetchall() )
    except: 
        continue
        #print(i,value)
    rstt_sales=rstt_sales.append(bb)
    
conn.close()
rstt_sales
print(rstt_sales.shape)
rstt_sales.head()

           CARD_NAME  COMPANY_ID             DEAL_DT  \
0       NH농협카드(9*9*)         494 2016-04-20 12:48:00   
1       NH농협카드(9*9*)         494 2016-04-25 13:42:00   
0         신한카드(3*8*)         687 2016-03-21 20:35:00   
1         기업BC(8*0*)         687 2016-03-04 17:22:00   
2       NH농협카드(5*5*)         687 2016-05-10 12:57:00   
3         기업BC(2*6*)         687 2016-03-01 18:14:00   
4         신한카드(2*1*)         687 2017-02-04 19:24:00   
5         기업BC(3592)         687 2016-09-12 15:04:36   
6       SC은행BC(3594)         687 2016-11-05 15:28:18   
0       KB국민체크(9*3*)        1036 2016-04-15 12:12:09   
1         우리카드(6921)        1036 2016-12-09 18:29:29   
2       현대카드 THE RED        1036 2016-11-09 18:38:48   
0         하나카드(7*0*)        1501 2016-09-29 12:07:27   
1         삼성카드(2497)        1501 2016-11-08 10:20:24   
0         삼성카드(7972)        1578 2016-05-27 00:54:00   
1       KB국민카드(2*3*)        1578 2016-02-01 21:27:00   
2       KB국민체크(0*3*)        1578 2016-07-08 20:0

,CARD_NAME,COMPANY_ID,DEAL_DT,EPOPCON_ID,PAYMENT,REG_DT
0,NH농협카드(9*9*),494,2016-04-20 12:48:00,eppcn15729a49e99bd03267fa61367,16000,2016-05-06 18:55:41
1,NH농협카드(9*9*),494,2016-04-25 13:42:00,eppcn15729a49e99bd03267fa61367,5000,2016-05-06 18:55:41
0,신한카드(3*8*),687,2016-03-21 20:35:00,eppcn157584b4999bd035ee606e8a5,83120,2016-06-09 01:44:59
1,기업BC(8*0*),687,2016-03-04 17:22:00,eppcn1572f238c99bd03267fa61570,40800,2016-05-08 20:32:04
2,NH농협카드(5*5*),687,2016-05-10 12:57:00,eppcn15732768b99bd03267fa617e3,31901,2016-05-11 09:02:36


In [51]:
#전체 유저별 150건수있는 놈만 걸러내기
temp = rstt_sales.groupby('EPOPCON_ID').count().reset_index()
temp=temp[['EPOPCON_ID','PAYMENT']]
temp['TOTAL']=temp['PAYMENT']
del temp['PAYMENT']
temp.head()
rstt_sales_150 = temp[temp['TOTAL']>150]
rstt_sales_150 = pd.merge(rstt_sales_150,rstt_sales, how='inner', on='EPOPCON_ID')
print(rstt_sales_150.shape)
rstt_sales_150.head() #464명

(102920, 7)


,EPOPCON_ID,TOTAL,CARD_NAME,COMPANY_ID,DEAL_DT,PAYMENT,REG_DT
0,eppcn1569f61d39d6b8d5f4d8e4242,175,하나카드(2*5*),60391,2016-05-06 20:08:00,12000,2016-05-06 20:10:08
1,eppcn1569f61d39d6b8d5f4d8e4242,175,신한카드(0*2*),104874,2016-05-22 15:32:00,5200,2016-05-22 15:33:26
2,eppcn1569f61d39d6b8d5f4d8e4242,175,하나카드(8*2*),112709,2016-05-15 17:54:00,15000,2016-05-15 17:55:47
3,eppcn1569f61d39d6b8d5f4d8e4242,175,,188537,2016-03-31 19:12:00,47000,2016-03-31 19:13:26
4,eppcn1569f61d39d6b8d5f4d8e4242,175,신한카드(0*2*),188537,2016-05-25 19:29:00,24000,2016-05-25 19:33:15


In [ ]:
#유저별 전체 결제건수

In [30]:
print(rstt_sales.shape)

(497260, 6)


In [ ]:
#유저별 pay > 2만원 건수

rstt_sales_150.groupby('EPOPCON_ID')#.filter(lambda x : x['PAYMENT']> 27414)

In [65]:
#del rstt_sales_150['USER_COMPANY_CNT']
# rstt_sales_150.head()
# rstt_sales_150['COMPANY_ID']=rstt_sales_150['COMPANY_ID_y']
# rstt_sales_150['COMPANY_CNT']=rstt_sales_150['COMPANY_ID_x']
del rstt_sales_150['COMPANY_ID_y']
del rstt_sales_150['COMPANY_ID_x']

In [ ]:
#유저별 상점수

In [66]:
#temp = 
#rstt_sales.groupby('EPOPCON_ID').COMPANY_ID.unique().shape[0]#.reset_index()
# temp=rstt_sales_150.groupby('EPOPCON_ID').COMPANY_ID.apply(lambda x: len(x.unique())).reset_index()#['COMPANY_ID']
# rstt_sales_150= pd.merge(temp,rstt_sales_150, how='inner',on='EPOPCON_ID')
#temp.head()
rstt_sales_150.head()

,EPOPCON_ID,TOTAL,CARD_NAME,DEAL_DT,PAYMENT,REG_DT,COMPANY_ID,COMPANY_CNT
0,eppcn1569f61d39d6b8d5f4d8e4242,175,하나카드(2*5*),2016-05-06 20:08:00,12000,2016-05-06 20:10:08,60391,70
1,eppcn1569f61d39d6b8d5f4d8e4242,175,신한카드(0*2*),2016-05-22 15:32:00,5200,2016-05-22 15:33:26,104874,70
2,eppcn1569f61d39d6b8d5f4d8e4242,175,하나카드(8*2*),2016-05-15 17:54:00,15000,2016-05-15 17:55:47,112709,70
3,eppcn1569f61d39d6b8d5f4d8e4242,175,,2016-03-31 19:12:00,47000,2016-03-31 19:13:26,188537,70
4,eppcn1569f61d39d6b8d5f4d8e4242,175,신한카드(0*2*),2016-05-25 19:29:00,24000,2016-05-25 19:33:15,188537,70


In [ ]:
#ratio 컬럼생성